In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/processed/processed.csv')

In [3]:
df.shape

(6100, 9)

In [4]:
df.sample(3)

,kms_driven,owner,location,mileage,power,price,brand,engine,age
2364,NaN,second,bangalore,40.0,20.7,75000.0,TVS,NaN,4
1477,18000.0,first,other,32.0,27.2,140000.0,Royal Enfield,500.0,5
2086,NaN,first,other,45.0,17.2,43400.0,TVS,180.0,7


In [5]:
X = df.drop('price',axis=1)
y = df['price']

In [6]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,38998.0,first,bangalore,40.0,19.80,Royal Enfield,350.0,10
1,24561.0,third,bangalore,35.0,25.00,KTM,200.0,9
2,NaN,first,other,40.0,20.21,TVS,NaN,1
3,19500.0,first,delhi,32.0,27.20,Royal Enfield,500.0,11
4,5947.0,first,chennai,53.0,19.00,Bajaj,NaN,4


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=11)

In [8]:
y = y.apply(np.log1p)

## Category Encoding

In [9]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,38998.0,first,bangalore,40.0,19.80,Royal Enfield,350.0,10
1,24561.0,third,bangalore,35.0,25.00,KTM,200.0,9
2,NaN,first,other,40.0,20.21,TVS,NaN,1
3,19500.0,first,delhi,32.0,27.20,Royal Enfield,500.0,11
4,5947.0,first,chennai,53.0,19.00,Bajaj,NaN,4


## Impute missing values

In [10]:
from sklearn.impute import KNNImputer

num_cols = [ col for col in X.columns if X[col].dtypes != 'object' ]
print(num_cols)

['kms_driven', 'mileage', 'power', 'engine', 'age']


In [11]:
imputer = KNNImputer(n_neighbors=7,weights='distance')

imputer.fit(X_train[num_cols])

# X_train[num_cols] = imputer.transform(X_train[num_cols])
# X_test[num_cols] = imputer.transform(X_test[num_cols])

X_train.loc[:][num_cols] = imputer.fit_transform(X_train[num_cols])
X_test.loc[:][num_cols] = imputer.transform(X_test[num_cols])

In [12]:
print('Missing Values in Train set',X_train.isnull().sum())
print('Missing Values in Test set',X_test.isnull().sum())

Missing Values in Train set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64
Missing Values in Test set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64


In [13]:
X_train.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
1264,32370.000000,first,other,45.0,13.0,Yamaha,150.0,7
1080,28662.891172,second,delhi,80.0,7.7,Bajaj,100.0,8
5122,11000.000000,first,other,35.0,19.8,Royal Enfield,350.0,5
2408,29490.000000,first,other,45.0,13.0,Yamaha,150.0,7
370,75000.000000,first,other,35.0,19.0,Bajaj,350.0,5


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler

category_transformer = ColumnTransformer([
    ("kms_driven_engine_min_max_scaler",MinMaxScaler(),[0,6,3,4]),
    ("owner_ordinal_enc",OrdinalEncoder(categories=[['fourth','third','second','first']],handle_unknown='ignore',dtype=np.int16),[1]),
    ("brand_location_ohe",OneHotEncoder(sparse=False,handle_unknown='error',drop='first',),[2,5]),
],remainder='passthrough')



## Model Building

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config

set_config(display='diagram')

In [16]:

def build_pipeline_with_estimator(estimator):
    return Pipeline([
    ('category_transformer',category_transformer),
    ('estimator',estimator),
])


In [17]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
# MAPE
def mape(targets, predictions):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1-((1-r2)*(n-1)/(n-k-1))

# Model performance check
def model_perf(model, inp, out,cross_val=True):

    y_pred = model.predict(inp)
    y_act = out.values

    cross_val_ = cross_val_score(model, inp,out,cv=10).mean() if cross_val else None
    

    return pd.DataFrame({
                "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
                "MAE": mean_absolute_error(y_act, y_pred),
                "MAPE": mape(y_act, y_pred),
                "R^2": r2_score(y_act, y_pred),
                "Adjusted R^2": adj_r2(inp, y_act, y_pred),
                "Cross Val Score (Mean)": cross_val_ if cross_val else None
           }, index=[0])

## LinearRegression

In [18]:
linear_regressor = build_pipeline_with_estimator(LinearRegression())

linear_regressor.fit(X_train,y_train)

print('Linear Regression Train Performance.\n')
print(model_perf(linear_regressor,X_train,y_train,True))

print('Linear Regression Test Performance.\n')
print(model_perf(linear_regressor,X_test,y_test,True))

Linear Regression Train Performance.

           RMSE           MAE      MAPE       R^2  Adjusted R^2  \
0  18111.515693  13224.347863  21.33513  0.800981      0.800632   

   Cross Val Score (Mean)  
0                0.798108  
Linear Regression Test Performance.

           RMSE          MAE       MAPE       R^2  Adjusted R^2  \
0  19800.274167  14091.17944  22.807015  0.769399      0.768182   

   Cross Val Score (Mean)  
0                 0.78942  


## RandomForest

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

model = build_pipeline_with_estimator(RandomForestRegressor())

params = {
    'estimator__criterion': ['mse','mae'],
    'estimator__n_estimators': [100,110,120,130],
    'estimator__max_depth': [5,10,15,20,25,30],
    'estimator__min_samples_split': range(2,20),
    'estimator__max_features': ['auto','sqrt','log2'],
}

model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('RandomForest Train Performance.\n')
print(model_perf(model,X_train,y_train,False))

print('RandomForest Test Performance.\n')
print(model_perf(model,X_test,y_test,False))

RandomForest Train Performance.

           RMSE         MAE       MAPE       R^2  Adjusted R^2  \
0  10921.497971  7980.95635  11.869051  0.927632      0.927505   

  Cross Val Score (Mean)  
0                   None  
RandomForest Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  15185.914704  10948.174361  16.426377  0.864356       0.86364   

  Cross Val Score (Mean)  
0                   None  


## XGBoost

In [20]:
from xgboost import XGBRegressor

xgboost = build_pipeline_with_estimator(XGBRegressor())

xgboost.fit(X_train,y_train)

print('xgboost Train Performance.\n')
print(model_perf(xgboost,X_train,y_train))

print('xgboost Test Performance.\n')
print(model_perf(xgboost,X_test,y_test))

xgboost Train Performance.

         RMSE          MAE      MAPE       R^2  Adjusted R^2  \
0  8167.73837  5791.911425  8.708507  0.959525      0.959454   

   Cross Val Score (Mean)  
0                0.879804  
xgboost Test Performance.

          RMSE           MAE      MAPE       R^2  Adjusted R^2  \
0  15564.82773  11024.974895  16.10422  0.857503      0.856751   

   Cross Val Score (Mean)  
0                0.820553  


## KNeighborsRegressor

In [21]:
from sklearn.neighbors import KNeighborsRegressor

model = build_pipeline_with_estimator(KNeighborsRegressor())

params = {
    'estimator__n_neighbors': [3,5,7,9],
    'estimator__algorithm': ['ball_tree','kd_tree','brute'],
    'estimator__leaf_size': [5,10,30,32,35]
}

model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('KNeighborsRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('KNeighborsRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

KNeighborsRegressor Train Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  14286.714217  10092.882279  15.063668  0.876163      0.875946   

   Cross Val Score (Mean)  
0                0.828097  
KNeighborsRegressor Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  17928.583286  12729.075972  18.780364  0.810935      0.809938   

   Cross Val Score (Mean)  
0                0.789219  


## GradientBoostingRegressor

In [22]:
from sklearn.ensemble import GradientBoostingRegressor

model = build_pipeline_with_estimator(GradientBoostingRegressor())

# params = {
#     'estimator__loss': ['ls','lad','huber','quantile'],
#     'estimator__learning_rate': [0.1,0.01,0.001],
#     'estimator__n_estimators': [100],
#     'estimator__criterion': ['friedman_mse','mse'],
# }

# model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('GradientBoostingRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('GradientBoostingRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

GradientBoostingRegressor Train Performance.

           RMSE          MAE       MAPE       R^2  Adjusted R^2  \
0  13403.229862  9574.418762  14.247485  0.891006      0.890815   

   Cross Val Score (Mean)  
0                0.875947  
GradientBoostingRegressor Test Performance.

           RMSE          MAE       MAPE       R^2  Adjusted R^2  \
0  14902.504679  10760.41641  16.052187  0.869372      0.868682   

   Cross Val Score (Mean)  
0                0.854162  
